In [1]:
# # install octave
# !sudo apt-get -qq update
# !sudo apt-get -qq install octave octave-signal liboctave-dev

# # # install oct2py that compatible with colab
# import os

# from pkg_resources import get_distribution

# os.system(f"pip install -qq"
#           f" ipykernel=={get_distribution('ipykernel').version}"
#           f" ipython=={get_distribution('ipython').version}"
#           f" tornado=={get_distribution('tornado').version}"
#           f" oct2py"
#           )

# # install packages
# !pip install -qq matpower matpowercaseframes

In [2]:
import oct2py

import matpower

print(f"oct2py version: {oct2py.__version__}")
print(f"matpower version: {matpower.__version__}")

oct2py version: 5.6.1
matpower version: 7.1.0.2.1.6


In [3]:
import numpy as np

from matpower import start_instance
from matpowercaseframes import CaseFrames

In [4]:
m = start_instance()  # runpf require oct2py / matlab

In [5]:
def runpf(mpc, m=None, verbose=False, inplace=True):
    if m is None:
        m = start_instance()
        SHUTDOWN = True
    else:
        SHUTDOWN = False

    # push value to octave client
    m.push('_mpc', mpc)

    # use octave native to run some commands
    # TODO: support mpoption
    m.eval("_mpopt = mpoption('verbose', 2);", verbose=verbose)
    m.eval("_r1 = runpf(_mpc, _mpopt);", verbose=verbose)

    # fech data to python (.eval is used because .pull is not working in acessing field)
    if not inplace:
        mpc = {}

    _extract_result(mpc, m, verbose=verbose)

    if SHUTDOWN:
        m.exit()

    return mpc

def runpf_with_pmax(mpc, m=m, verbose=True, inplace=True):
    # base run
    mpc = runpf(mpc, m=m, verbose=True, inplace=True)  # using runopf wrapper
    cf = CaseFrames(mpc)

    # get slack value
    slack_bus_index = cf.bus.index[cf.bus['BUS_TYPE'].astype(int) == 3]
    slack_gen_index = cf.gen.index[cf.gen['GEN_BUS'].astype(int) == slack_bus_index[0]]  # 0 to extract value
    slack_gen_power = cf.gen.loc[slack_gen_index, 'PG']

    # TODO: support multiple generator on slack_bus, detect sum and aggregate
    if (slack_gen_power < cf.gen.loc[slack_gen_index, 'PMAX']).any(axis=None):
        # early return slack got no problem
        # return mpc
        pass

    # get priority list
    priority_list = get_priority_list(cf)
    current_priority_list_index = 0  # python integer

    # fix slack
    n_gen = len(cf.gen)
    while (slack_gen_power > cf.gen.loc[slack_gen_index, 'PMAX']).any(axis=None):
        cf.gen.loc[slack_gen_index, 'PG'] = cf.gen.loc[slack_gen_index, 'PMAX']

        # swap slack bus
        cf.bus.loc[slack_bus_index, 'BUS_TYPE'] = 2  # became PV
        current_priority_list_index += 1
        slack_gen_index = [priority_list[current_priority_list_index]]
        slack_bus_index = cf.gen.loc[slack_gen_index, 'GEN_BUS'].values
        cf.bus.loc[slack_bus_index, 'BUS_TYPE'] = 3  # became slack

        # runpf
        mpc = cf.to_dict()
        mpc = runpf(mpc, m=m, verbose=True, inplace=True)  # using runopf wrapper
        cf = CaseFrames(mpc)

        # get slack power
        slack_gen_power = cf.gen.loc[slack_gen_index, 'PG']

        # break all generator already pmax
        if current_priority_list_index > n_gen:
            break

    return mpc

def get_priority_list(cf):
    # NOTE: return pandas .loc index
    n_const_cost = cf.gencost['NCOST'].iloc[0]

    n_gen = len(cf.gen)
    cost = np.zeros(n_gen)
    for i in range(int(n_const_cost)):
        cost += cf.gencost[f'COST_{i}'] * cf.gen['PMAX'] ** i
    heat_rate = (cost / cf.gen['PMAX'])

    priority_list = heat_rate.sort_values().index
    return priority_list

def _extract_result(mpc, m, verbose=False):
    mpc['baseMVA'] = m.eval('_r1.baseMVA;', verbose=verbose)
    mpc['version'] = m.eval('_r1.version;', verbose=verbose)
    mpc['bus'] = m.eval('_r1.bus;', verbose=verbose)
    mpc['gen'] = m.eval('_r1.gen;', verbose=verbose)
    mpc['branch'] = m.eval('_r1.branch;', verbose=verbose)
    mpc['gencost'] = m.eval('_r1.gencost;', verbose=verbose)

In [6]:
mpc = m.loadcase('case9', verbose=False)
cf = CaseFrames(mpc)
cf.gen.loc[1, 'PMAX'] = 60
cf.gen.loc[2, 'PMAX'] = 170
cf.gen

,GEN_BUS,PG,QG,QMAX,QMIN,VG,MBASE,GEN_STATUS,PMAX,PMIN,...,PC2,QC1MIN,QC1MAX,QC2MIN,QC2MAX,RAMP_AGC,RAMP_10,RAMP_30,RAMP_Q,APF
1,1.0,72.3,27.03,300.0,-300.0,1.040,100.0,1.0,60.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,163.0,6.54,300.0,-300.0,1.025,100.0,1.0,170.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,85.0,-10.95,300.0,-300.0,1.025,100.0,1.0,270.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
mpc = cf.to_dict()
mpc = runpf(mpc, m=m, verbose=True, inplace=True)  # using runpf wrapper

cf = CaseFrames(mpc)
cf.gen


MATPOWER Version 7.1, 08-Oct-2020 -- AC Power Flow (Newton)

 it    max P & Q mismatch (p.u.)
----  ---------------------------
  0         1.630e+00
  1         1.875e-01
  2         2.147e-03
  3         3.421e-07
  4         1.837e-14
Newton's method power flow (power balance, polar) converged in 4 iterations.

Converged in 0.02 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     500.0        -900.0 to 900.0
Generators         3     On-line Capacity       500.0        -900.0 to 900.0
Committed Gens     3     Generation (actual)    319.6              22.8
Loads              3     Load                   315.0             115.0
  Fixed            3       Fixed                315.0             115.0
  Dispatchable     0       Dispatchable          -0

,GEN_BUS,PG,QG,QMAX,QMIN,VG,MBASE,GEN_STATUS,PMAX,PMIN,...,PC2,QC1MIN,QC1MAX,QC2MIN,QC2MAX,RAMP_AGC,RAMP_10,RAMP_30,RAMP_Q,APF
1,1.0,71.641021,27.045924,300.0,-300.0,1.040,100.0,1.0,60.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,163.000000,6.653660,300.0,-300.0,1.025,100.0,1.0,170.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,85.000000,-10.859709,300.0,-300.0,1.025,100.0,1.0,270.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
mpc = cf.to_dict()
mpc = runpf_with_pmax(mpc, m=m, verbose=True, inplace=True)  # using runpf_with_pmax wrapper

cf = CaseFrames(mpc)
cf.gen


MATPOWER Version 7.1, 08-Oct-2020 -- AC Power Flow (Newton)

 it    max P & Q mismatch (p.u.)
----  ---------------------------
  0         1.837e-14
Converged!

Converged in 0.00 seconds
|     System Summary                                                           |

How many?                How much?              P (MW)            Q (MVAr)
---------------------    -------------------  -------------  -----------------
Buses              9     Total Gen Capacity     500.0        -900.0 to 900.0
Generators         3     On-line Capacity       500.0        -900.0 to 900.0
Committed Gens     3     Generation (actual)    319.6              22.8
Loads              3     Load                   315.0             115.0
  Fixed            3       Fixed                315.0             115.0
  Dispatchable     0       Dispatchable          -0.0 of -0.0      -0.0
Shunts             0     Shunt (inj)             -0.0               0.0
Branches           9     Losses (I^2 * Z)         4.64       

,GEN_BUS,PG,QG,QMAX,QMIN,VG,MBASE,GEN_STATUS,PMAX,PMIN,...,PC2,QC1MIN,QC1MAX,QC2MIN,QC2MAX,RAMP_AGC,RAMP_10,RAMP_30,RAMP_Q,APF
1,1.0,60.00000,28.607360,300.0,-300.0,1.040,100.0,1.0,60.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,170.00000,8.239483,300.0,-300.0,1.025,100.0,1.0,170.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,90.18877,-10.186462,300.0,-300.0,1.025,100.0,1.0,270.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
